
RF (random forest) — это множество деревьев решений. При решении задачии регрессии ответы усредняются, при решении задачи
классификаци приоритет отдаются максимамально найденному параметру.

Суть случайного леса сводится к следующему: выбирается подвыборка обучающей выборки размера samplesize  – по ней строится дерево
(для каждого дерева — своя подвыборка). Для построения каждого ответлвения в дереве просматриваем max_features случайных
признаков (для каждого нового ответлвения — свои случайные признаки). Выбираем наилучшие признак и расщепление по нему
(по заранее заданному критерию). Дерево строится, как правило, до исчерпания выборки (пока в листьях не останутся представители
только одного класса), но в современных реализациях есть параметры, которые ограничивают высоту дерева, число объектов в 
листьях и число объектов в подвыборке, при котором проводится расщепление. Такая схема построения соответствует главному
принципу ансамблирования (построению алгоритма машинного обучения на базе нескольких, в данном случае решающих деревьев): 
базовые алгоритмы должны быть хорошими и разнообразными (поэтому каждое дерево строится на своей обучающей выборке и 
при выборе расщеплений есть элемент случайности).

Основные параметры:

n_estimators -Число деревьев.
Чем больше деревьев, тем лучше качество, но время настройки и работы RF также пропорционально увеличиваются. Обратите внимание, что часто при увеличении n_estimators качество на обучающей выборке повышается (может даже доходить до 100%), а качество на тесте выходит на асимптоту (можно прикинуть, скольких деревьев Вам достаточно).

max_features - Число признаков для выбора расщепления
График качества на тесте от значения этого праметра унимодальный, на обучении он строго возрастает. При увеличении max_features увеличивается время построения леса, а деревья становятся «более однообразными». По умолчанию он равен sqrt(n) в задачах классификации и n/3 в задачах регрессии. Это ключевой параметр. Его настраивают в первую очередь (при достаточном числе деревьев в лесе).

min_samples_split - Минимальное число объектов, при котором выполняется расщепление
Допускается оставлять этот параметр по умолчанию.

max_depth - Максимальная глубина деревьев
Ясно, что чем меньше глубина, тем быстрее строится и работает RF. При увеличении глубины резко возрастает качество на обучении, но и на контроле оно, как правило, увеличивается. Рекомендуется использовать максимальную глубину (кроме случаев, когда объектов слишком много и получаются очень глубокие деревья, построение которых занимает значительное время). При использовании неглубоких деревьев изменение параметров, связанных с ограничением числа объектов в листе и для деления, не приводит к значимому эффекту (листья и так получаются «большими»). Неглубокие деревья рекомендуют использовать в задачах с большим числом шумовых объектов (выбросов).

criterion - Критерий расщепления
По смыслу это очень важный параметр, но по факту здесь нет вариантов выбора. В библиотеке sklearn для регрессии реализованы два критерия: “mse” и “mae”, соответствуют функциям ошибки, которые они минимизируют. В большинстве задач используется mse.  Для классификации реализованы критерии “gini” и “entropy”, которые соответствуют классическим критериям расщепления: Джини и энтропийному. 

В sklearn-реализации случайного леса нет параметра samplesize, который регламентирует, из скольких объектов делать подвыборку для построения каждого дерева. Зачастую он и не нужен, поскольку оптимально выбирать из всей выборки. Также рекомендуется выбирать подвыборку с возвращением: bootstrap=True ( вообщем и целом это и есть пресловутый бэггинг — bootstrap aggregating).

По умолчанию в sklearn-овских методах n_jobs=1, т.е. случайный лес строится на одном процессоре. Если Вы хотите существенно ускорить построение, используйте n_jobs=-1 (строить на максимально возможном числе процессоров). Для построения воспроизводимых экспериментов используйте предустановку генератора псевдослучайных чисел: random_state.

Помимо прочего при   построении леса random forest параллельно может вычисляться т.н. oob-оценка качества алгоритма (которая очень точная и получается не в ущерб разделения на обучение/тест), oob-ответы алгоритмы (ответы, которые выдавал бы алгоритм на обучающей выборке, если бы «обучался не на ней»), оцениваются важности признаков. 

In [1]:
# Подргужаем библиотеки

import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import RandomizedSearchCV

In [2]:

# Подружаем изначальный набора данных
data = pd.read_csv(r'C:\Тест\Python_Обучение\data\titanic.csv')

# Выбираем обучающий набор, удаляя не нужные столбцы
x = data.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin"], axis=1)

#преобразуем строковые значения в числовые. Для проведения дальльнейшего анализа.
x = pd.get_dummies(x)

# Все пропуски в обучаеющем наборе заполним медианным значением
x = x.fillna({'Age': x.Age.median()})

# Целевой набор. В данном случае остался ли жив человек.
y = data.Survived

In [3]:
# Разбиваем на тестовые и тренировочные признаки и ответы в соотношении 67 к 33.
# random_state - задает зерно случайности при выборке строк. 
# Если его не задать, то при каждом запуске в тест и train будут попадать разные строки.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state = 42)


Random Forest - создание нескольких деревьев, но каждому дереву выделена часть признаков (колонок) и часть данных (строк)
Всем деревья переданы одинаковые параметры обучения (глубина, размер сплита и пр.). Результатом является усредненная 
вероятность деревьев, отработавших по своей части столбцов и строк.
Причем, чем больше деревьев - тем лучше.

Случайный лес решает сразу 2 проблемы обучения деревьев:
* множеств невысоких деревьев - решает проблему переобучения
* перебор порядка колонок - проблему неоптимальности жадного алгоритма, т.к. выбирает порядок фич почти случайно.

In [4]:
# Зададим параметры модели
RF = RandomForestClassifier(n_estimators=10,
                            max_depth = 3,
                            
                           )
# Обучим модель
RF.fit(X_train, y_train) 

# Предскажем значения на тестовой выборке
RF.score(X_test, y_test)


0.6153846153846154

63 % точности, совсем не то чего хотелось бы получить от алгоритма машинного обучения. Попробуем добавить параметров и использовать метод GridSearchCV. Который подберет наилучшие параметры при обучении.                                                                                                                                                                         

In [5]:
clf_rf = RandomForestClassifier()

#n_estimators - число деревьев
parameters = {'n_estimators': [10, 20, 30], 'max_depth': [2,5,7,10]}
grid_search_cv_clf = GridSearchCV(clf_rf, parameters, cv=5)
grid_search_cv_clf.fit(X_train, y_train)

grid_search_cv_clf.best_params_ # {'max_depth': 5, 'n_estimators': 30}
best_clf = grid_search_cv_clf.best_estimator_
grid_search_cv_clf.best_params_
best_clf.score(X_test, y_test) 

0.8269230769230769

In [ ]:
82% значительно более хороший результат нежели чем 62%.

In [6]:
RF = RandomForestClassifier()

#n_estimators - число деревьев
parameters = {'n_estimators': [10, 20, 30], 
              'max_depth': [2,5,7,10],
             }
grid_search_RF = GridSearchCV(RF,
                              parameters,
                              cv=5)
grid_search_RF.fit(X_train, y_train)

grid_search_RF.best_params_ 
best_RF = grid_search_RF.best_estimator_
grid_search_RF.best_params_
best_RF.score(X_test, y_test) 

0.7692307692307693

76 % уже лучше. Теперь попробуем еще один метод подбора наиболее оптимальных параметров RandomizedSearchCV. 

In [7]:
RF = RandomForestClassifier()

#n_estimators - число деревьев
parameters = {'n_estimators': [10, 20, 30], 
              'max_depth': [2,5,7,10],
             }
grid_search_RF = RandomizedSearchCV(RF,
                              parameters,
                              cv=5)
grid_search_RF.fit(X_train, y_train)

grid_search_RF.best_params_ # {'max_depth': 5, 'n_estimators': 30}
best_RF = grid_search_RF.best_estimator_
grid_search_RF.best_params_
best_RF.score(X_test, y_test)

0.7692307692307693

Как видим результат идентичен. Но у данного метода есть одно важное приемущество. Он работает несколько более быстрее чем 
GridSearchCV. На больших выборках это особенно заметно.